In [3]:
import cv2
print("OpenCV version:", cv2.__version__)


OpenCV version: 4.10.0


# code 1

In [2]:
import cv2

pixels = cv2.imread('team.jpg')
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

gray = cv2.cvtColor(pixels, cv2.COLOR_BGR2GRAY)

bboxes = classifier.detectMultiScale(gray, 
                                     scaleFactor=1.1, 
                                     minNeighbors=9, 
                                     minSize=(30, 30)
                                      )


for (x, y, width, height) in bboxes:
    cv2.rectangle(pixels, (x, y), (x + width, y + height), (255, 0, 0), 2)
    
cv2.imshow('Detected Objects', pixels)

cv2.waitKey(0)
cv2.destroyAllWindows()


# code 2

In [3]:
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv2.imread('team.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor=1.05,
    minNeighbors=3,
    minSize=(30, 30),
    maxSize=(300, 300)
)

for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

cv2.imshow('img', img)
cv2.waitKey(0)

cv2.destroyAllWindows()
